# Chapter 2, Types
Provides a more exhaustive explanation of types and traits in Rust, including how the compiler reasons about them, their features and restrictions, and a number of advanced applications.

## Types in memory
- Types are used to interpret bits of memory.
- Details about memory layout affect both correctness and performance (i.e.: _misaligned accesses_)
- Due to hardware costraints all values must be at least byte-aligned

In [29]:
println!("{0: <10} {1: <10}", std::any::type_name::<u8>(), std::mem::align_of::<u8>());
println!("{0: <10} {1: <10}", std::any::type_name::<u16>(), std::mem::align_of::<u16>());
println!("{0: <10} {1: <10}", std::any::type_name::<u32>(), std::mem::align_of::<u32>());
println!("{0: <10} {1: <10}", std::any::type_name::<u64>(), std::mem::align_of::<u64>());
println!("{0: <10} {1: <10}", std::any::type_name::<u128>(), std::mem::align_of::<u128>());

struct Foo {
    f1: u8,
    f2: u64,
}
println!("{0: <10} {1: <10}", std::any::type_name::<Foo>(), std::mem::align_of::<Foo>());

u8         1         
u16        2         
u32        4         
u64        8         
u128       8         
ctx::Foo   8         


Note: the order of the fields in a struct influeces the final size in memory (due to padding)
It is possible to force a particular memory layout:
- `#[repr(Rust)]`: default one. The compiler is free to reorder fields in order to optimize
- `#[repr(C)]`: force C memory layout. The fields order is preserved.
- `#[repr(transparent)]`: force a struct containing only 1 non-zero sized type to be laid out identically to the inner type
- `#[repr(packed)]`: no padding between struct fields
- `#[repr(align(n))]`: force a larger alignement than required
Rust still doesn't have a stable ABI. So, it is currently suggested to specify `#[repr(C)]` to cross the binary boundaries.

Not all types have a size known at compile time (e.g.: `str`, `[i32]`, `&dyn`). Everywhere a known size is required there is an implicit `Sized` trait bound (with an opt-out policiy). _Fat pointers_ allow to referenced unsized types: they contains the runtime size along with the memory location.
## Traits and Trait Bounds
- _monomorphization_: the compiler replaces generic types and functions with concrete instances for each type T (only code actually used)
- _static dispatch_: every call of a generic function is implemented with a jump the instance for the current concrete type
- _dynamic dispatch_: allow to call a trait method on a generic type when the concrete type is known only at runtime. Implemented with a _vtable_. Requires the `dyn` keyword (add a performance overhead)
- _trait object_: a type that implements a trait + its vtable. Stored as two pointers (one to the data struct, one to the vtable)
- only _object-safe_ traits can be turned into a trait object. A trait is object-safe when no generic method uses `Self`
### Generic traits
- Associated types should be preferred to generic traits but only allow one implementation per type

Example:

In [12]:
struct Type {}

trait Foo {
    type AssociatedType;
}

impl Foo for Type {
    type AssociatedType = i32;
}

// Will not compile
/*impl Foo for Type {
    type Bar = String;
}*/

trait GenericFoo<T> {

}

impl GenericFoo<i32> for Type {}
impl GenericFoo<String> for Type {}

### Coherence and the Orphan Rule
- _coherence_: the property that there is at most one implementation of a trait for any given type.
- Rust enforces coherence by simply refusing to compile programs that contain conflicting implementations.
- prevent "dependency hell" problems where multiple crates contain conflicting `impl`
- the orphan rule says that you can implement a trait for a type only if the trait **or** the type is local to your crate
- there are exceptions to the Orphan Rule
### Trait bounds
- `#[derive(Trait)]` usually desugar into `impl Trait for Foo<T> where T: Trait`
### Marker traits
- traits with empty bodies which allow to capture semantic requirements.
- often are also _auto-traits_: the compiler automatically implements them
- _marker types_: unit types that help to prevent API misuses
## Existential types
- `async fn` functions or when return type is `impl Trait have an existential return type: the caller can not know the actual returned type
- allow you to perform zero-cost type erasure

Example:

In [13]:
pub trait File {
    fn content(&self) -> String;
}
  
impl File for String {
    fn content(&self) -> String {
        self.clone()
    }
}
  
fn open() -> impl File {
    "this is not a file".to_string()
}

fn read(file: impl File) -> () {
    println!("{:?}", file.content());
}
  
{
  let file = open();
  read(file);
}

"this is not a file"


()